In [67]:
# !pip install -r requirements.txt

In [68]:
import ifcopenshell
import rdflib
import pyshacl
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef, OWL, XSD
from rdflib.namespace import DC, FOAF
from ifcopenshell.util.element import get_psets

# Open the IFC file
ifc_file = ifcopenshell.open('./data/Atlas_8_floor.ifc')

In [70]:
g = Graph()
SCHEMA = Namespace("https://schema.org/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
OTLB = Namespace("https://schema.example.ch/otl/base")
OTL = Namespace("https://schema.example.ch/otl/")
DATA = Namespace("https://data.example.ch/")

In [71]:
g = Graph()
g.bind("schemaorg", SCHEMA)
g.bind("geo", GEO)
g.bind("otlb", OTLB)
g.bind("otl", OTL)

element_type = 'IfcStair'
elements = ifc_file.by_type('IfcRoot')

relevant_pset = "Pset_StairCommon"
relevant_object = "STRAIGHT_RUN_STAIR"

for element in elements: # Loop over the elements -> If defined in OTL -> Add to graph
    element_info = element.get_info()
    predef_type = element_info.get('PredefinedType')
    
    if predef_type != relevant_object:
        continue

    pset = get_psets(element)[relevant_pset]

    # TODO: Here Mapping for system reference to object class predef_type
    g.add((DATA[element.GlobalId], RDF.type, OTL["Stair"])) # Write the object to the graph

    for key, value in pset.items():
        if isinstance(value, bool):
            datatype = XSD.boolean
        elif isinstance(value, int):
            datatype = XSD.integer
        elif isinstance(value, float):
            datatype = XSD.float
        else:
            datatype = XSD.string

        g.add((DATA[element.GlobalId], OTL[key], Literal(value, datatype=datatype)))

print(g.serialize(format='turtle'))

@prefix otl: <https://schema.example.ch/otl/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://data.example.ch/0TWrSBEvj7$BuYv3P4M6fG> a otl:Stair ;
    otl:IsExternal false ;
    otl:NosingLength "0.0"^^xsd:float ;
    otl:NumberOfRiser "0.0"^^xsd:float ;
    otl:NumberOfTreads "0.0"^^xsd:float ;
    otl:Reference "190mm max riser 250mm going"^^xsd:string ;
    otl:id 299073 .

<https://data.example.ch/0qT631TWf0IwT1wFys3KdH> a otl:Stair ;
    otl:IsExternal false ;
    otl:NosingLength "0.0"^^xsd:float ;
    otl:NumberOfRiser "22.0"^^xsd:float ;
    otl:NumberOfTreads "21.0"^^xsd:float ;
    otl:Reference "190mm max riser 250mm going"^^xsd:string ;
    otl:RiserHeight "0.181818181817031"^^xsd:float ;
    otl:TreadLength "0.25"^^xsd:float ;
    otl:TreadLengthAtInnerSide "0.25"^^xsd:float ;
    otl:TreadLengthAtOffset "0.25"^^xsd:float ;
    otl:id 298862 .

<https://data.example.ch/2G8FWepbLFNPaY9uTwv0w$> a otl:Stair ;
    otl:IsExternal false ;
    otl:NosingLength "0.

In [74]:
# Create a graph object
shape_graph = Graph()
shape_graph.parse("./otl/build_profile.ttl", format="turtle")
# print(shape_graph.serialize(format="turtle"))

<Graph identifier=N299f9eaf53ae46da9cd27df421988b87 (<class 'rdflib.graph.Graph'>)>

In [73]:
conforms, results_graph, results_text = pyshacl.validate(g, shacl_graph=shape_graph)
print(conforms)
print(results_text)

False
Validation Report
Conforms: False
Results (3):
Constraint Violation in MinInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:minInclusive Literal("20", datatype=xsd:integer) ; sh:path otl:NumberOfRiser ]
	Focus Node: <https://data.example.ch/2XFVGLalX4ifUdAEsElx54>
	Value Node: Literal("0.0", datatype=xsd:float)
	Result Path: otl:NumberOfRiser
	Message: Value is not >= Literal("20", datatype=xsd:integer)
Constraint Violation in MinInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:minInclusive Literal("20", datatype=xsd:integer) ; sh:path otl:NumberOfRiser ]
	Focus Node: <https://data.example.ch/2G8FWepbLFNPaY9uTwv0w$>
	Value Node: Literal("0.0", datatype=xsd:float)
	Result Path: otl:NumberOfRiser
	Message: Value is no